In [33]:
import cv2
import fingerprint_feature_extractor
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from PIL import Image, ImageFilter

In [14]:

def binarize_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    return binary_image


def skeletonize_image(binary_image):
    skeleton = cv2.ximgproc.thinning(binary_image, thinningType=cv2.ximgproc.THINNING_ZHANGSUEN)
    return skeleton


def find_special_points(skeleton):
    points = []
    for i in range(skeleton.shape[0]):
        for j in range(skeleton.shape[1]):
            if skeleton[i, j] == 255:
                points.append((i, j))
    return points

def compare_fingerprints(points1, points2):
    match_score = 0
    for point in points1:
        if point in points2:
            match_score += 1
    return match_score

In [25]:
def show_compare_images(first_image_path, second_image_path):
    img1 = open(first_image_path, 'rb').read()
    img2 = open(second_image_path, 'rb').read()
    wi1 = widgets.Image(value=img1, format='jpg', width=300, height=400)
    wi2 = widgets.Image(value=img2, format='jpg', width=300, height=400)
    wid = widgets.HBox([wi1, wi2])
    display(wid)

In [26]:
def test_image_yourself():
    binary_image1 = binarize_image('res/print_leha.png')
    skeleton1 = skeletonize_image(binary_image1)
    points1 = find_special_points(skeleton1)

    binary_image2 = binarize_image('res/print_leha.png')
    skeleton2 = skeletonize_image(binary_image2)
    points2 = find_special_points(skeleton2)

    match_score = compare_fingerprints(points1, points2)
    show_compare_images('res/print_leha.png', 'res/print_leha.png')
    print(f'Степень соответствия: {match_score}')


test_image_yourself()

Степень соответствия: 18902


In [27]:
def test_different():
    binary_image1 = binarize_image('res/print_leha.png')
    skeleton1 = skeletonize_image(binary_image1)
    points1 = find_special_points(skeleton1)

    binary_image2 = binarize_image('res/print_oleg.png')
    skeleton2 = skeletonize_image(binary_image2)
    points2 = find_special_points(skeleton2)

    match_score = compare_fingerprints(points1, points2)
    show_compare_images('res/print_leha.png', 'res/print_oleg.png')
    print(f'Степень соответствия: {match_score}')


test_different()

Степень соответствия: 2952


In [28]:
def blur_image(input_path, output_path):
    img = Image.open(input_path)
    blurred_img = img.filter(ImageFilter.GaussianBlur(radius=2))
    blurred_img.save(output_path)


def test_blurred_yourself():
    binary_image1 = binarize_image('res/print_leha.png')
    skeleton1 = skeletonize_image(binary_image1)
    points1 = find_special_points(skeleton1)

    blur_image("res/print_leha.png", "res/blurred.png")

    binary_image2 = binarize_image("res/blurred.png")
    skeleton2 = skeletonize_image(binary_image2)
    points2 = find_special_points(skeleton2)

    match_score = compare_fingerprints(points1, points2)
    show_compare_images('res/print_leha.png', 'res/blurred.png')
    print(f'Степень соответствия: {match_score}')


test_blurred_yourself()

Степень соответствия: 8904


In [31]:
def lighting_image(img, coef):
    img = np.array(img) / 255  # Нормализация значений пикселей к диапазону [0, 1]
    img = img + img * coef  # Увеличение яркости на коэффициент
    img[img > 1] = 1  # Ограничение значений пикселей до 1
    lighted_img = Image.fromarray((img * 255).astype(np.uint8))
    lighted_img.save("res/lighten.png")


def test_lighten_yourself():
    binary_image1 = binarize_image('res/print_leha.png')
    skeleton1 = skeletonize_image(binary_image1)
    points1 = find_special_points(skeleton1)

    img = Image.open('res/print_leha.png')
    img_array = np.array(img)
    lighting_image(img_array, 0.9)

    binary_image2 = binarize_image('res/lighten.png')
    skeleton2 = skeletonize_image(binary_image2)
    points2 = find_special_points(skeleton2)

    match_score = compare_fingerprints(points1, points2)
    show_compare_images('res/print_leha.png', 'res/lighten.png')
    print(f'Степень соответствия: {match_score}')


test_lighten_yourself()   

Степень соответствия: 14735


In [35]:

def find_by_features(image_path):
    img = cv2.imread(image_path, 0)
    FeaturesTerminations, FeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(
        img,
        spuriousMinutiaeThresh=10,
        invertImage=False,
        showResult=True,
        saveResult=True)
    terminations = [(feature.locX, feature.locY, feature.Orientation, feature.Type) for feature in FeaturesTerminations]
    return [(feature.locX, feature.locY, feature.Orientation, feature.Type) for feature in FeaturesBifurcations]

def compare_minutiae(minutiae1, minutiae2):
    scores = []
    for m1 in minutiae1:
        for m2 in minutiae2:
            score = np.linalg.norm(m1 - m2) # Евклидово расстояние
            scores.append(score)
    return scores

def compare_images():
    minutiae1 = find_by_features("res/print_leha.png")
    minutiae2 = find_by_features("res/print_leha.jpg")
    print(compare_minutiae(minutiae1, minutiae2))
    
#compare_images()

TypeError: '>' not supported between instances of 'NoneType' and 'int'